In [3]:
import sys
from pathlib import Path
import matplotlib.pyplot as plt

project_root = Path("/dmj/fizmed/mmarzec/licencjat_neuro/magisterka")
sys.path.insert(0, str(project_root / "src"))

In [4]:
from preprocessing.frame_exporter import EEGFrameExporter #type:ignore

In [5]:
data = EEGFrameExporter.load_frames("demoDTF/1/frames/eyes_closed_frames_20251210_225525.npz")
frames = data['frames']
channels = data['channel_names']
sfreq = data['sfreq']